In [2]:
# rm -r valid
# ! unzip ~/Symbols/train.zip -d ~/
# ! unzip ~/Symbols/valid.zip -d ~/
# ! unzip ~/Symbols/test.zip -d ~/
# ls
# Unzipping the corresponding files for use in the JupyterLab

In [36]:
ls

Classification.ipynb  test.pickle  train.pickle  valid/
Models/               train/       train.zip     valid.zip


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras import optimizers
from keras import applications
from keras.models import Model
from keras.callbacks import History
from tensorflow.keras.callbacks import ReduceLROnPlateau
import keras

Using TensorFlow backend.


In [3]:
from tensorflow.keras.applications.resnet import preprocess_input
# from tensorflow.keras.applications.inception_v3 import preprocess_input
# from tensorflow.keras.applications.densenet import preprocess_input

In [44]:
# loading the training data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                #    shear_range=0.2,
                                  #  zoom_range=0.4,
                                  #  rotation_range=30,
                                   )

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Here the training files are accessed. Place here the directory of the train and validation files
train_generator = train_datagen.flow_from_directory(
        '/home/jovyan/Symbols/train',
        target_size=(45, 45),
        class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    '/home/jovyan/Symbols/valid',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

Found 26925 images belonging to 60 classes.
Found 11540 images belonging to 60 classes.


In [45]:
from tensorflow.keras.applications import ResNet152, ResNet50, InceptionV3, DenseNet121, DenseNet201
# Importing the corresponding models above for training

In [46]:
model = Sequential()
# create the base pre-trained model
model.add(ResNet50(input_shape=(45, 45,3), weights='imagenet', include_top=False))
# add a global spatial average pooling layer
model.add(GlobalAveragePooling2D())
# let's add a fully-connected layer
model.add(Dense(1024, activation='relu'))
# and a logistic layer -- let's say we have 60 classes
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [47]:
# specify training loss function 
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [48]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose = 1)

# Specify the name of the filepath where the model is to be saved. Here it is called resnet50e30 as it uses a pretrained ResNet50 under 30 epochs
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/jovyan/Symbols/Models/resnet50e30',
    monitor='val_loss',
    save_weights_only=False,
    save_best_only=True)

model.fit(x=train_generator, validation_data=validation_generator, epochs = 30, callbacks=[reduce_lr,model_checkpoint])

Epoch 1/30
842/842 [==============================] - 75s 90ms/step - loss: 0.6737 - accuracy: 0.8310 - val_loss: 6.0021 - val_accuracy: 0.6894 - lr: 1.0000e-04
Epoch 2/30
842/842 [==============================] - 71s 85ms/step - loss: 0.2060 - accuracy: 0.9424 - val_loss: 3.1686 - val_accuracy: 0.8209 - lr: 1.0000e-04
Epoch 3/30
842/842 [==============================] - 75s 89ms/step - loss: 0.1450 - accuracy: 0.9598 - val_loss: 2.4374 - val_accuracy: 0.8873 - lr: 1.0000e-04
Epoch 4/30
842/842 [==============================] - 71s 84ms/step - loss: 0.1082 - accuracy: 0.9689 - val_loss: 1.2019 - val_accuracy: 0.9123 - lr: 1.0000e-04
Epoch 5/30
842/842 [==============================] - 67s 79ms/step - loss: 0.0859 - accuracy: 0.9745 - val_loss: 0.8149 - val_accuracy: 0.9282 - lr: 1.0000e-04
Epoch 6/30
842/842 [==============================] - 76s 91ms/step - loss: 0.0753 - accuracy: 0.9792 - val_loss: 0.5149 - val_accuracy: 0.9410 - lr: 1.0000e-04
Epoch 7/30
842/842 [==============

In [19]:
# Here the model may be evaluated. First, import the test files
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    '/home/jovyan/Symbols/test',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

Found 5082 images belonging to 61 classes.


In [20]:
# test_generator.classes
# test_generator.class_indices
# test_generator.filenames

In [21]:
# The models top5 accuracies can be found below
top1 = 0
top2 = 0
top3 = 0
top4 = 0
top5 = 0
predict = model.predict(test_generator)

for i in range(len(predict)):
  result = np.argsort(-predict[i])
  truth = test_generator.classes[i]
  if truth in result[:5]: top5 += 1
  if truth in result[:4]: top4 += 1
  if truth in result[:3]: top3 += 1
  if truth in result[:2]: top2 += 1
  if truth in result[:1]: top1 += 1

top1 = top1/len(predict)
top2 = top2/len(predict)
top3 = top3/len(predict)
top4 = top4/len(predict)
top5 = top5/len(predict)
print(top1, top2, top3, top4, top5)

0.1123573396300669 0.15072805981896892 0.1721763085399449 0.18929555293191658 0.2028728846910665


In [18]:
from tensorflow import keras
model = keras.models.load_model('/home/jovyan/Symbols/Models/resnet50e30')

In [17]:
from tensorflow.keras.applications.resnet import preprocess_input
# from tensorflow.keras.applications.inception_v3 import preprocess_input
# from tensorflow.keras.applications.densenet import preprocess_input

In [50]:
# loading training data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                #    shear_range=0.2,
                                  #  zoom_range=0.4,
                                  #  rotation_range=30,
                                   )

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Specify the directory of the training and validation files
train_generator = train_datagen.flow_from_directory(
        '/home/jovyan/Symbols/train',
        target_size=(45, 45),
        class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    '/home/jovyan/Symbols/valid',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

Found 26925 images belonging to 60 classes.
Found 11540 images belonging to 60 classes.


In [51]:
from tensorflow.keras.applications import ResNet152, ResNet50, InceptionV3, DenseNet121, DenseNet201

In [52]:
model = Sequential()
# create the base pre-trained model
model.add(ResNet152(input_shape=(45, 45,3), weights='imagenet', include_top=False))
# add a global spatial average pooling layer
model.add(GlobalAveragePooling2D())
# let's add a fully-connected layer
model.add(Dense(1024, activation='relu'))
# and a logistic layer -- let's say we have 60 classes
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [53]:
# specify training loss function 
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [54]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose = 1)
# Add the directory in which the model is saved
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/jovyan/Symbols/Models/resnet152e30',
    monitor='val_loss',
    save_weights_only=False,
    save_best_only=True)

model.fit(x=train_generator, validation_data=validation_generator, epochs = 30, callbacks=[reduce_lr,model_checkpoint])

Epoch 1/30
842/842 [==============================] - 176s 209ms/step - loss: 0.7001 - accuracy: 0.8240 - val_loss: 178.3596 - val_accuracy: 0.7617 - lr: 1.0000e-04
Epoch 2/30
842/842 [==============================] - 168s 199ms/step - loss: 0.2375 - accuracy: 0.9357 - val_loss: 45.5554 - val_accuracy: 0.8843 - lr: 1.0000e-04
Epoch 3/30
842/842 [==============================] - 174s 206ms/step - loss: 0.1563 - accuracy: 0.9561 - val_loss: 25.4327 - val_accuracy: 0.8242 - lr: 1.0000e-04
Epoch 4/30
842/842 [==============================] - 170s 202ms/step - loss: 0.1236 - accuracy: 0.9655 - val_loss: 6.0164 - val_accuracy: 0.9006 - lr: 1.0000e-04
Epoch 5/30
842/842 [==============================] - 90s 107ms/step - loss: 0.1137 - accuracy: 0.9709 - val_loss: 14.6381 - val_accuracy: 0.9091 - lr: 1.0000e-04
Epoch 6/30
842/842 [==============================] - 91s 108ms/step - loss: 0.0977 - accuracy: 0.9735 - val_loss: 19.3339 - val_accuracy: 0.9217 - lr: 1.0000e-04
Epoch 7/30
842/842

In [9]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Specify test images directory
test_generator = test_datagen.flow_from_directory(
    '/home/jovyan/Symbols/test',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

Found 5082 images belonging to 61 classes.


In [10]:
# test_generator.classes
# test_generator.class_indices
# test_generator.filenames

In [24]:
top1 = 0
top2 = 0
top3 = 0
top4 = 0
top5 = 0
predict = model.predict(test_generator)

for i in range(len(predict)):
  result = np.argsort(-predict[i])
  truth = test_generator.classes[i]
  if truth in result[:5]: top5 += 1
  if truth in result[:4]: top4 += 1
  if truth in result[:3]: top3 += 1
  if truth in result[:2]: top2 += 1
  if truth in result[:1]: top1 += 1

top1 = top1/len(predict)
top2 = top2/len(predict)
top3 = top3/len(predict)
top4 = top4/len(predict)
top5 = top5/len(predict)
print(top1, top2, top3, top4, top5)

0.11097992916174734 0.14167650531286896 0.16843762298307752 0.1887052341597796 0.2081857536402991


In [23]:
from tensorflow import keras
model = keras.models.load_model('/home/jovyan/Symbols/Models/resnet152e30')

In [6]:
# Import the appropriate preprocess input for the model
# from tensorflow.keras.applications.resnet import preprocess_input
# from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input

In [56]:
# loading training data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                #    shear_range=0.2,
                                  #  zoom_range=0.4,
                                  #  rotation_range=30,
                                   )

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Specify the directory of the training and validation files
train_generator = train_datagen.flow_from_directory(
        '/home/jovyan/Symbols/train',
        target_size=(45, 45),
        class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    '/home/jovyan/Symbols/valid',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

Found 26925 images belonging to 60 classes.
Found 11540 images belonging to 60 classes.


In [57]:
from tensorflow.keras.applications import ResNet152, ResNet50, InceptionV3, DenseNet121, DenseNet201

In [58]:
model = Sequential()
# create the base pre-trained model
model.add(DenseNet121(input_shape=(45, 45,3), weights='imagenet', include_top=False))
# add a global spatial average pooling layer
model.add(GlobalAveragePooling2D())
# let's add a fully-connected layer
model.add(Dense(1024, activation='relu'))
# and a logistic layer -- let's say we have 60 classes
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [59]:
# specify training loss function 
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [60]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose = 1)
# Specify the directory in which the model is saved
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/jovyan/Symbols/Models/densenet121e30',
    monitor='val_loss',
    save_weights_only=False,
    save_best_only=True)

model.fit(x=train_generator, validation_data=validation_generator, epochs = 30, callbacks=[reduce_lr,model_checkpoint])

Epoch 1/30
842/842 [==============================] - 120s 143ms/step - loss: 0.8038 - accuracy: 0.7969 - val_loss: 0.3955 - val_accuracy: 0.9117 - lr: 1.0000e-04
Epoch 2/30
842/842 [==============================] - 57s 68ms/step - loss: 0.2411 - accuracy: 0.9297 - val_loss: 0.4510 - val_accuracy: 0.9271 - lr: 1.0000e-04
Epoch 3/30
842/842 [==============================] - 52s 61ms/step - loss: 0.1713 - accuracy: 0.9498 - val_loss: 0.4408 - val_accuracy: 0.9412 - lr: 1.0000e-04
Epoch 4/30
842/842 [==============================] - 113s 134ms/step - loss: 0.1354 - accuracy: 0.9594 - val_loss: 0.3694 - val_accuracy: 0.9464 - lr: 1.0000e-04
Epoch 5/30
842/842 [==============================] - 58s 69ms/step - loss: 0.1130 - accuracy: 0.9651 - val_loss: 0.3901 - val_accuracy: 0.9409 - lr: 1.0000e-04
Epoch 6/30
842/842 [==============================] - 116s 137ms/step - loss: 0.0986 - accuracy: 0.9699 - val_loss: 0.3685 - val_accuracy: 0.9438 - lr: 1.0000e-04
Epoch 7/30
842/842 [========

In [7]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# Specify the test images directory
test_generator = test_datagen.flow_from_directory(
    '/home/jovyan/Symbols/test',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

Found 5082 images belonging to 61 classes.


In [8]:
# test_generator.classes
# test_generator.class_indices
# test_generator.filenames

In [9]:
top1 = 0
top2 = 0
top3 = 0
top4 = 0
top5 = 0
predict = model.predict(test_generator)

for i in range(len(predict)):
  result = np.argsort(-predict[i])
  truth = test_generator.classes[i]
  if truth in result[:5]: top5 += 1
  if truth in result[:4]: top4 += 1
  if truth in result[:3]: top3 += 1
  if truth in result[:2]: top2 += 1
  if truth in result[:1]: top1 += 1

top1 = top1/len(predict)
top2 = top2/len(predict)
top3 = top3/len(predict)
top4 = top4/len(predict)
top5 = top5/len(predict)
print(top1, top2, top3, top4, top5)

0.1127508854781582 0.15348288075560804 0.18850846123573398 0.21251475796930341 0.23415977961432508


In [2]:
from tensorflow import keras
model = keras.models.load_model('/home/jovyan/Symbols/Models/densenet121e30')

In [61]:
# Import the corresponding preprocess input

# from tensorflow.keras.applications.resnet import preprocess_input
# from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input

In [62]:
# loading training data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                #    shear_range=0.2,
                                  #  zoom_range=0.4,
                                  #  rotation_range=30,
                                   )

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Specify the directory of the training and validation files
train_generator = train_datagen.flow_from_directory(
        '/home/jovyan/Symbols/train',
        target_size=(45, 45),
        class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    '/home/jovyan/Symbols/valid',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

Found 26925 images belonging to 60 classes.
Found 11540 images belonging to 60 classes.


In [63]:
from tensorflow.keras.applications import ResNet152, ResNet50, InceptionV3, DenseNet121, DenseNet201

In [64]:
model = Sequential()
# create the base pre-trained model
model.add(DenseNet201(input_shape=(45, 45,3), weights='imagenet', include_top=False))
# add a global spatial average pooling layer
model.add(GlobalAveragePooling2D())
# let's add a fully-connected layer
model.add(Dense(1024, activation='relu'))
# and a logistic layer -- let's say we have 60 classes
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [65]:
# specify training loss function 
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [66]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose = 1)
# Specify the filepath for the model to be saved
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/jovyan/Symbols/Models/densenet201e30',
    monitor='val_loss',
    save_weights_only=False,
    save_best_only=True)

model.fit(x=train_generator, validation_data=validation_generator, epochs = 30, callbacks=[reduce_lr,model_checkpoint])

Epoch 1/30
842/842 [==============================] - 193s 229ms/step - loss: 0.6836 - accuracy: 0.8303 - val_loss: 0.4583 - val_accuracy: 0.9166 - lr: 1.0000e-04
Epoch 2/30
842/842 [==============================] - 88s 105ms/step - loss: 0.2231 - accuracy: 0.9380 - val_loss: 0.6642 - val_accuracy: 0.9336 - lr: 1.0000e-04
Epoch 3/30
842/842 [==============================] - 192s 228ms/step - loss: 0.1661 - accuracy: 0.9511 - val_loss: 0.4288 - val_accuracy: 0.9365 - lr: 1.0000e-04
Epoch 4/30
842/842 [==============================] - 90s 107ms/step - loss: 0.1354 - accuracy: 0.9618 - val_loss: 0.7661 - val_accuracy: 0.9365 - lr: 1.0000e-04
Epoch 5/30
842/842 [==============================] - 190s 225ms/step - loss: 0.1157 - accuracy: 0.9666 - val_loss: 0.3823 - val_accuracy: 0.9477 - lr: 1.0000e-04
Epoch 6/30
842/842 [==============================] - 90s 106ms/step - loss: 0.1004 - accuracy: 0.9711 - val_loss: 0.4532 - val_accuracy: 0.9500 - lr: 1.0000e-04
Epoch 7/30
842/842 [=====

In [28]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    '/home/jovyan/Symbols/test',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

Found 5082 images belonging to 61 classes.


In [24]:
# test_generator.classes
# test_generator.class_indices
# test_generator.filenames

In [30]:
top1 = 0
top2 = 0
top3 = 0
top4 = 0
top5 = 0
predict = model.predict(test_generator)

for i in range(len(predict)):
  result = np.argsort(-predict[i])
  truth = test_generator.classes[i]
  if truth in result[:5]: top5 += 1
  if truth in result[:4]: top4 += 1
  if truth in result[:3]: top3 += 1
  if truth in result[:2]: top2 += 1
  if truth in result[:1]: top1 += 1

top1 = top1/len(predict)
top2 = top2/len(predict)
top3 = top3/len(predict)
top4 = top4/len(predict)
top5 = top5/len(predict)
print(top1, top2, top3, top4, top5)

0.11078315623770169 0.15387642660369932 0.170995670995671 0.18752459661550572 0.20031483667847305


In [1]:
from tensorflow import keras
model = keras.models.load_model('/home/jovyan/Symbols/Models/densenet201e30')